In [4]:
import pandas as pd
import numpy as np
import pickle

actualRates = pd.read_csv('../beast/supplement/discreteTraits/results/discreteTraits_2016-10-26_actualRates.csv', index_col=0, header=0).T

s = actualRates.sum(axis=0) # Series of the sum of each row
maxRate = s.max() # Max row sum value

def normalize(x, total):
    if total == 0:
        return x
    else:									
        return np.float64(x) / np.float64(total)

for host, row in actualRates.iterrows(): # Iterate over rows   
    total = row.sum() 
    extra = maxRate - total
    # Add to diagonals so all row totals are equal
    row = row.fillna(extra) 
    
    # Normalize so each row sums to 1
    actualRates.loc[host,:] = row.apply(normalize, args=(maxRate,)) 

eigenValues, eigenVectors  = np.linalg.eig(actualRates.T) # NOT SORTED
rearrangedEvalsVecs = sorted(zip(eigenValues,eigenVectors.T), key=lambda x: x[0].real, reverse=True) #thanks stack overflow
principleEigenVector = rearrangedEvalsVecs[0][1]
total = float(principleEigenVector.sum())

principleEigenVector = pd.Series([ float(x) / total for x in principleEigenVector ], index = actualRates.index.values)
principleEigenVector.sort_values(inplace=True)
print principleEigenVector


Human                  0.001054
Chimp_pts              0.006239
Chimp_ptt              0.006782
Colobus_Monkey         0.009223
Mandrill               0.014447
Sooty_Mangabey         0.015430
DeBrazzas_Monkey       0.017116
Western_red_colobus    0.017693
Gorilla                0.018284
Red-capped_Mangabey    0.066527
Red-eared_Guenon       0.070091
Vervet                 0.091364
Tantalus_Monkey        0.092028
Patas_monkey           0.168551
Baboon                 0.188008
Sabaeus_Monkey         0.217163
dtype: float64


/Library/Python/2.7/site-packages/ipykernel/__main__.py:28: ComplexWarning: Casting complex values to real discards the imaginary part
/Library/Python/2.7/site-packages/ipykernel/__main__.py:30: ComplexWarning: Casting complex values to real discards the imaginary part


In [6]:
principleEigenVector = { host: score for host, score in principleEigenVector.iteritems() }
pickle.dump(principleEigenVector, open('../figures/sizes.p', 'wb'))